In [1]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json

In [2]:
census_zips = pd.read_csv('/Volumes/data/yelp_geo_zips.csv')
yelp_with_dollars = pd.read_csv('/Volumes/data/folium/examples/Yelp_Clean.csv')
census_zips_bounds = gpd.read_file('/Volumes/data/cb_2017_us_zcta510_500k/cb_2017_us_zcta510_500k.shp')

In [3]:
zip_stats = pd.read_csv('/Volumes/data/IRS_zipcodes_clean.csv')

In [4]:
census_zips_bounds = census_zips_bounds[['ZCTA5CE10','geometry']]
census_zips_bounds.columns = ['zip','geometry']
census_zips_bounds['zip'] = census_zips_bounds['zip'].astype(int)
census_frame = pd.DataFrame(census_zips_bounds)

In [5]:
m2 = folium.Map([34, -112], zoom_start=10)

In [6]:
def chloroplether(specific_zips):
    specific_frame = yelp_with_dollars[yelp_with_dollars['zip'].isin(specific_zips)]
    specific_frame.reset_index(drop=True,inplace=True)
    
    filtered_zips = census_frame[census_frame['zip'].isin(specific_zips)]
    filtered_zips.reset_index(drop=True,inplace=True)
    
    filtered_census = census_zips_bounds[census_zips_bounds['zip'].isin(filtered_zips['zip'])]
    filtered_census.reset_index(drop=True,inplace=True)
    filtered_census.crs = {'init' :'epsg:4326'}
    
    zip_stats.columns = ['zip',1,2,3,4,5,6]
    zip_stats_tidy = filtered_census.merge(zip_stats,on='zip',how='left')[['zip',1,2,3,4,5,6]]
    
    zip_stats['total'] = zip_stats[[1,2,3,4,5,6]].sum(axis=1)
    
    zip_stats['affluence'] = ((zip_stats[1] * 1) 
                          + (zip_stats[2] * 2) 
                          + (zip_stats[3] * 3) 
                          + (zip_stats[4] * 4) 
                          + (zip_stats[4] * 4)
                          + (zip_stats[5] * 5)
                          + (zip_stats[6] * 6) / zip_stats['total'])
    
    zip_stats_supertidy = zip_stats[['zip','affluence']]
    filtered_census = filtered_census.merge(zip_stats_supertidy,on='zip',how='left')
    
    filtered_census['zip'] = filtered_census['zip'].astype(str)
    tidy_census = filtered_census[['zip','affluence']]
    
    tidy_census.columns = ['regionid','stat']
    filtered_census.columns = ['regionid','geometry','stat']
    filtered_census.crs = {'init' :'epsg:4326'}
    
    
    
    chloropleth = folium.Choropleth(
    geo_data=filtered_census,
    data=tidy_census,
    columns=['regionid', 'stat'],
    key_on='feature.properties.regionid',
    fill_color='YlGnBu',
    highlight=True,
    name='Neighborhood Affluency',
    #bins=[3, 4, 9, 11]
    )
    
    return chloropleth

In [7]:
chloroplether([85255,85377,85266,85331,85263,85262]).add_to(m2)

In [8]:
zipcode = 85377

In [9]:
import branca


html = f"""
    <h4>{zipcode}</h4>

    """

In [10]:
folium.Marker([33.82430958796856, -111.915357039909], popup=html).add_to(m2)

This gets the center of a polygon

In [22]:
str(census_zips_bounds['geometry'][0].centroid)

'POINT (-88.19359122978537 33.07370545099048)'

In [11]:
m2

In [ ]:
#m.save(os.path.join('results', 'html_popups.html'))

m

In [ ]:
m

In [17]:
def point_plotter(specific_zips):
    specific_frame = yelp_with_dollars[yelp_with_dollars['zip'].isin(specific_zips)]
    specific_frame.reset_index(drop=True,inplace=True)
    
    specific_frame1 = specific_frame.loc[specific_frame['dollar_sign'] == 1]
    specific_frame2 = specific_frame.loc[specific_frame['dollar_sign'] == 2]
    specific_frame3 = specific_frame.loc[specific_frame['dollar_sign'] == 3]
    specific_frame4 = specific_frame.loc[specific_frame['dollar_sign'] == 4]
    
    name1 = list(specific_frame1["name"])
    lat1 = list(specific_frame1["latitude"])
    lon1 = list(specific_frame1["longitude"])

    name2 = list(specific_frame2["name"])
    lat2 = list(specific_frame2["latitude"])
    lon2 = list(specific_frame2["longitude"])

    name3 = list(specific_frame3["name"])
    lat3 = list(specific_frame3["latitude"])
    lon3 = list(specific_frame3["longitude"])

    name4 = list(specific_frame4["name"])
    lat4 = list(specific_frame4["latitude"])
    lon4 = list(specific_frame4["longitude"])
    
    fg1 = folium.FeatureGroup(name="1 Dollar Sign")
    fg2 = folium.FeatureGroup(name="2 Dollar Signs")
    fg3 = folium.FeatureGroup(name="3 Dollar Signs")
    fg4 = folium.FeatureGroup(name="4 Dollar Signs")
    
    
    for lt, ln, nm in zip(lat1, lon1, name1):
        fg1.add_child(folium.CircleMarker(location=[lt, ln], radius = 1, popup= nm + ", " + str(1)+" price",
        fill_color='red', fill=True,  color = 'red', fill_opacity=0.7))

    for lt, ln, nm in zip(lat2, lon2, name2):
        fg2.add_child(folium.CircleMarker(location=[lt, ln], radius = 1, popup= nm + ", " + str(2)+" price",
        fill_color='orange', fill=True,  color = 'orange', fill_opacity=0.7))

    for lt, ln, nm in zip(lat3, lon3, name3):
        fg3.add_child(folium.CircleMarker(location=[lt, ln], radius = 1, popup= nm + ", " + str(3)+" price",
        fill_color='yellow', fill=True,  color = 'yellow', fill_opacity=0.7))

    for lt, ln, nm in zip(lat4, lon4, name4):
        fg4.add_child(folium.CircleMarker(location=[lt, ln], radius = 1, popup= nm + ", " + str(4)+" price",
        fill_color='green', fill=True,  color = 'green', fill_opacity=0.7))
        
    m2.add_child(fg1)
    m2.add_child(fg2)
    m2.add_child(fg3)
    m2.add_child(fg4)

In [18]:
point_plotter([85255,85377,85266,85331,85263,85262])

In [19]:
m2